In [ ]:
!pip install \
    accelerate==1.5.2 \
    huggingface-hub==0.27.0 \
    safetensors==0.5.0 \
    sentence-transformers==3.3.1 \
    torch==2.6.0 \
    transformers==4.50.3 \
    langchain-community \
    langchain-huggingface

In [5]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_community.embeddings import SentenceTransformerEmbeddings

/opt/conda/lib/python3.10/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-05-22 12:05:55.168116: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already b

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
# read the data
df = pd.read_parquet('ml_research_assistant/data/papers_with_abstract.parquet')
df.shape

(20286, 5)

In [10]:
df.head(2)

,paper_year,paper_url,paper_title,paper_author,paper_abstract
0,2023,https://papers.nips.cc/paper_files/paper/2023/...,Modelling Cellular Perturbations with the Spar...,"Michael Bereket, Theofanis Karaletsos",Generative models of observations under interv...
1,2023,https://papers.nips.cc/paper_files/paper/2023/...,Cross-Episodic Curriculum for Transformer Agents,"Lucy Xiaoyang Shi, Yunfan Jiang, Jake Grigsby,...","We present a new algorithm, Cross-Episodic Cur..."


In [11]:
df.tail(2)

,paper_year,paper_url,paper_title,paper_author,paper_abstract
20284,1987,https://papers.nips.cc/paper_files/paper/1987/...,Minkowski-r Back-Propagation: Learning in Conn...,"Stephen Hanson, David Burr",Many connectionist learning models are impleme...
20285,1987,https://papers.nips.cc/paper_files/paper/1987/...,Analysis and Comparison of Different Learning ...,J. Bernasconi,We investigate the behavior of different learn...


In [12]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    x = torch.randn(10).cuda()
    print(x)

PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
tensor([ 1.4846,  1.5773,  0.0996,  0.0618, -0.8350, -0.6634,  0.3206, -0.3603,
         0.9237,  0.6285], device='cuda:0')


In [14]:
!ls -la /kaggle/input/gemma-2/transformers/gemma-2-9b-it/2

total 18071664
drwxr-xr-x 3 nobody nogroup          0 Sep  3  2024 .
drwxr-xr-x 3 root   root          4096 May 22 11:47 ..
-rw-r--r-- 1 nobody nogroup       1661 Sep  3  2024 .gitattributes
-rw-r--r-- 1 nobody nogroup      25789 Sep  3  2024 README.md
-rw-r--r-- 1 nobody nogroup        857 Sep  3  2024 config.json
-rw-r--r-- 1 nobody nogroup        173 Sep  3  2024 generation_config.json
-rw-r--r-- 1 nobody nogroup 4903351912 Sep  3  2024 model-00001-of-00004.safetensors
-rw-r--r-- 1 nobody nogroup 4947570872 Sep  3  2024 model-00002-of-00004.safetensors
-rw-r--r-- 1 nobody nogroup 4962221464 Sep  3  2024 model-00003-of-00004.safetensors
-rw-r--r-- 1 nobody nogroup 3670322200 Sep  3  2024 model-00004-of-00004.safetensors
-rw-r--r-- 1 nobody nogroup      39072 Sep  3  2024 model.safetensors.index.json
-rw-r--r-- 1 nobody nogroup        636 Sep  3  2024 special_tokens_map.json
-rw-r--r-- 1 nobody nogroup   17525357 Sep  3  2024 tokenizer.json
-rw-r--r-- 1 nobody nogroup    4241003 Sep  

In [16]:
# load google gemma2-9b-it in 8-bit
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

config_8bit = BitsAndBytesConfig(load_in_8bit=True)
model_path = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, quantization_config=config_8bit,
                                            device_map=0)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [17]:
hf_pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10
)
local_llm = HuggingFacePipeline(pipeline=hf_pipe)

Device set to use cuda:0


In [20]:
# we compare retrieval on the basis of a base model and a model that we finetuned
encoder_original = SentenceTransformerEmbeddings(model_name='distilbert-base-uncased')

In [26]:
# the finetuned model is in huggingface hub
from huggingface_hub import login
login(token="YOUR_TOKEN")

In [27]:
encoder_finetuned = SentenceTransformerEmbeddings(model_name='abhishekmle/distilbert-base-uncased-finetuned')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/42.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/500 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [28]:
encoder_finetuned

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='abhishekmle/distilbert-base-uncased-finetuned', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [29]:
from typing import List, Tuple, Dict, Any

from langchain.llms.base import LLM
from langchain.embeddings.base import Embeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

import re

In [30]:
def index_documents(self, docs: List[str]) -> None:
    documents = [Document(page_content=d) for d in docs]
    self.store = FAISS.from_documents(documents, self.embeddings)

def index_documents_to_stores(
    docs: List[str], 
    embeddings_dict: Dict[str, Embeddings]
) -> Dict[str, FAISS]:
    """
    Indexes the documents into separate FAISS stores, one for each embedding model.
    """
    documents = [Document(page_content=d) for d in docs]
    
    stores = {}
    for model_name, embedding in embeddings_dict.items():
        stores[model_name] = FAISS.from_documents(documents, embedding)
    
    return stores

In [33]:
embeddings_dict = {
    "base_encoder": encoder_original,
    "finetuned_encoder": encoder_finetuned
}

docs = df['paper_abstract'].tolist()
print(len(docs))

stores = index_documents_to_stores(docs, embeddings_dict)
# Now you can access stores["openai"] and stores["hf"]

20286


### qualitative comparison between the un-tuned and the fine-tuned version of the encoder

#### 1st example

In [40]:
stores['base_encoder'].similarity_search_with_score(query='Mathematical understanding of weight initialization methods in neural networks.', k=3)

[(Document(id='a2bfbcb3-b7f9-4870-9494-44a3e695b108', metadata={}, page_content='This paper addresses the issue of numerical computation in machine learning domains based on similarity metrics, such as kernel methods, spectral techniques and Gaussian processes. It presents a general solution strategy based on Krylov subspace iteration and fast N-body learning methods. The experiments show significant gains in computation and storage on datasets arising in image segmentation, object detection and dimensionality reduction. The paper also presents theoretical bounds on the stability of these methods.'),
  19.048574),
 (Document(id='235c3469-be19-4957-ac1f-d1ee6cdefb11', metadata={}, page_content='A short account is  given of various  investigations of neural  network  properties,  beginning  with  the  classic  work of McCulloch  & Pitts.  Early work on neurodynamics and statistical mechanics, analogies with  magnetic materials, fault tolerance via parallel distributed processing,  memory

##### first two results of this un-tuned model are not relevant to the query, third result is somewhat relevant

In [41]:
stores['finetuned_encoder'].similarity_search_with_score(query='Mathematical understanding of weight initialization methods in neural networks.', k=3)

[(Document(id='a47bfaa5-a116-4d16-8373-51408d8e34ee', metadata={}, page_content='Current deep neural networks(DNNs) can easily overfit to biased training data with corrupted labels or class imbalance. Sample re-weighting strategy is commonly used to alleviate this issue by designing a weighting function mapping from training loss to sample weight, and then iterating between weight recalculating and classifier updating. Current approaches, however, need manually pre-specify the weighting function as well as its additional hyper-parameters. It makes them fairly hard to be generally applied in practice due to the significant variation of proper weighting schemes relying on the investigated problem and training data. To address this issue, we propose a method capable of adaptively learning an explicit weighting function directly from data. The weighting function is an MLP with one hidden layer, constituting a universal approximator to almost any continuous functions, making the method able

##### all three results of the fine-tuned model are relevant to the query

#### 2nd example

In [43]:
stores['base_encoder'].similarity_search_with_score(query='Are there any pitfalls of using cross-validation for model selection?', k=3)

[(Document(id='4d131d8c-36a0-49e8-ab69-762492f0ca33', metadata={}, page_content='We consider situations where training data is abundant and computing resources are comparatively scarce. We argue that suitably designed on- line learning algorithms asymptotically outperform any batch learning algorithm. Both theoretical and experimental evidences are presented.'),
  15.779577),
 (Document(id='0731a26a-d32a-4922-bd9a-544fabe24eb4', metadata={}, page_content='We study the effectiveness of neural sequence models for premise selection in automated theorem proving, a key bottleneck for progress in formalized mathematics. We propose a two stage approach for this task that yields good results for the premise selection task on the Mizar corpus while avoiding the hand-engineered features of existing state-of-the-art models. To our knowledge, this is the first time deep learning has been applied  theorem proving on a large scale.'),
  16.483631),
 (Document(id='8a2538fc-09fe-4968-aeb6-0aa51825d49b

##### all three results are irrelevant

In [44]:
stores['finetuned_encoder'].similarity_search_with_score(query='Are there any pitfalls of using cross-validation for model selection?', k=3)

[(Document(id='143b7a98-801b-4b09-bac7-c63650f6647a', metadata={}, page_content='Model selection is linked to model assessment, which is the problem of comparing different models, or model parameters, for a speciﬁc learning task. For supervised learning, the standard practical technique is cross- validation, which is not applicable for semi-supervised and unsupervised settings. In this paper, a new model assessment scheme is introduced which is based on a notion of stability. The stability measure yields an upper bound to cross-validation in the supervised case, but extends to semi-supervised and unsupervised problems. In the experimental part, the performance of the stability measure is studied for model order se- lection in comparison to standard techniques in this area.'),
  84.082214),
 (Document(id='f0e00af4-1922-4a37-9d46-dbb661785714', metadata={}, page_content='We address the problem of non-parametric multiple model comparison: given $l$\ncandidate models, decide whether each c

##### pitfalls of cross-validation is highly specific query which the model cannot answer, most probably because no such information is present in the abstracts, however 1st and 3rd results are relevant to cross-validation, and 2nd result talks about model selection which is indirectly related to cross-validation

### for quantitative evaluation of the retrieval results from the two models, we use LLM as a judge to provide us with relevancy scores

In [108]:
def extract_number(text: str) -> int:
    # Try to find an isolated number between 0 and 10
    match = re.search(r"Score: (\d+(\.\d+)?)?$", text)
    if match:
        try:
            int_match = match.group(1)
        except:
            int_match = 0
        return int_match
    return None

class RAGEvaluator:

    def __init__(self, llm, store):
        self.llm = llm
        self.store = store

    def retrieve(self, query: str, k: int = 3) -> List[Tuple[str, float]]:
        results = self.store.similarity_search_with_score(query, k=k)
        return [(doc.page_content, score) for doc, score in results]

    def score_with_llm(self, query: str, passage: str) -> Tuple[int, int]:
        prompt_relevance = f"""You are an expert in text understanding. You have to determine how relevant is a document to a given question on a scale of integers 0-10. While giving the score, please think very carefully whether the given document is actually a good match to the question on various parameters of relevance. The input will be given to as a question and the document. You have to return your score from 0-10. Please do not output any additional tokens beyond the numerical score.
        Question: {query}
        Document: {passage}
        Score: """
        prompt_binary = f"""You are an expert in text understanding. You have to determine how relevant is a document to a given question with a relevant match being 1 and irrelevant match being 0. While giving the score, please think very carefully whether the given document is actually a good match to the question on various parameters of relevance. The input will be given to as a question and the document. You have to return your score which can only be either 0 or 1. Please do not output any additional tokens beyond the numerical score.
        Question: {query}
        Document: {passage}
        Score: """
        #import pdb
        #pdb.set_trace()
        prompt_num = prompt_relevance.format(query=query, passage=passage)
        prompt_bin = prompt_binary.format(query=query, passage=passage)
    
        rel_resp = str(local_llm(prompt_num)).strip()
        bin_resp = str(local_llm(prompt_bin)).strip()
    
        try:
            rel_resp_int = int(extract_number(text=rel_resp))
        except:
            rel_resp_int = 0
        try:
            bin_resp_int = int(extract_number(text=bin_resp))
        except:
            bin_resp_int = 0
    
        return rel_resp_int, bin_resp_int

    def overlap_metrics(self, query: str, passage: str) -> Dict[str, int]:
        def ngrams(tokens: List[str], n: int) -> List[Tuple[str, ...]]:
            return [tuple(tokens[i : i + n]) for i in range(len(tokens) - n + 1)]

        q_tokens = query.split()
        p_tokens = passage.split()
        overlaps = {
            'word_overlap': len(set(q_tokens) & set(p_tokens)),
            '2gram_overlap': len(set(ngrams(q_tokens, 2)) & set(ngrams(p_tokens, 2))),
            '3gram_overlap': len(set(ngrams(q_tokens, 3)) & set(ngrams(p_tokens, 3)))
        }
        return overlaps

    def evaluate_retrieval(self, query: str, k: int = 3, use_llm: bool = True) -> Dict[str, Any]:
        passages, retrieval_scores = zip(*self.retrieve(query, k))
        rel_scores, bin_scores, overlaps = [], [], []
        for p in passages:
            rel, bin_ = self.score_with_llm(query, p) if use_llm else (0, 0)
            rel_scores.append(rel)
            bin_scores.append(bin_)
            overlaps.append(self.overlap_metrics(query, p))

        return {
            'query': query,
            'passages': passages,
            'retrieval_scores': retrieval_scores,
            'rel_scores': rel_scores,
            'bin_scores': bin_scores,
            'overlap': overlaps,
            'precision@k': self.precision_at_k(bin_scores, k)
        }

    def precision_at_k(self, bin_scores: List[int], k: int) -> float:
        return sum(bin_scores[:k]) / k

In [109]:
# Initialize evaluator
evaluator_base = RAGEvaluator(local_llm, stores['base_encoder'])
evaluator_finetuned = RAGEvaluator(local_llm, stores['finetuned_encoder'])

In [97]:
x1, y1 = zip(*evaluator_base.retrieve(query = "Which learning algorithms have been used to model brain activity in humans?", k=3))

In [98]:
x1

("While Attention has come to be an important mechanism in deep learning, there remains limited intuition for why it works so well. Here, we show that Transformer Attention can be closely related under certain data conditions to Kanerva's Sparse Distributed Memory (SDM), a biologically plausible associative memory model. We confirm that these conditions are satisfied in pre-trained GPT2 Transformer models. We discuss the implications of the Attention-SDM map and provide new computational and biological interpretations of Attention.",
 'The  goal  in  this  work  has  been  to  identify  the  neuronal  elements  of the cortical column that are most likely to support  the learning  of nonlinear associative  maps.  We show that  a  particular style  of  network learning algorithm based on locally-tuned  receptive fields  maps  naturally  onto cortical  hardware,  and  gives  coherence  to  a  variety of features  of cortical anatomy,  physiology,  and  biophysics  whose  relations to lear

In [99]:
y1

(16.56313, 17.2448, 17.45152)

In [100]:
evaluator_finetuned.evaluate_retrieval("Which learning algorithms have been used to model brain activity in humans?")

{'query': 'Which learning algorithms have been used to model brain activity in humans?',
 'passages': ("Neuroscience studies of human decision-making abilities commonly involve\nsubjects completing a decision-making task while BOLD signals are\nrecorded using fMRI. Hypotheses are tested about which brain regions\nmediate the effect of past experience, such as rewards, on future\nactions. One standard approach to this is model-based fMRI data\nanalysis, in which a model is fitted to the behavioral data, i.e., a\nsubject's choices, and then the neural data are parsed to find brain\nregions whose BOLD signals are related to the model's internal\nsignals. However, the internal mechanics of such purely behavioral\nmodels are not constrained by the neural data, and therefore might miss\nor mischaracterize aspects of the brain. To address this limitation, we\nintroduce a new method using recurrent neural network models that are\nflexible enough to be jointly fitted to the behavioral and neura

In [101]:
evaluator_base.evaluate_retrieval("Which learning algorithms have been used to model brain activity in humans?")

{'query': 'Which learning algorithms have been used to model brain activity in humans?',
 'passages': ("While Attention has come to be an important mechanism in deep learning, there remains limited intuition for why it works so well. Here, we show that Transformer Attention can be closely related under certain data conditions to Kanerva's Sparse Distributed Memory (SDM), a biologically plausible associative memory model. We confirm that these conditions are satisfied in pre-trained GPT2 Transformer models. We discuss the implications of the Attention-SDM map and provide new computational and biological interpretations of Attention.",
  'The  goal  in  this  work  has  been  to  identify  the  neuronal  elements  of the cortical column that are most likely to support  the learning  of nonlinear associative  maps.  We show that  a  particular style  of  network learning algorithm based on locally-tuned  receptive fields  maps  naturally  onto cortical  hardware,  and  gives  coherence  t

### send a list of queries to both the encoders to evaluate which one has better retrieval

In [102]:
# make a list of questions for which you want literature review

query_list = [
    "What are some breakthroughs in theoretical understanding of neural networks?",
    "Explain how random forest has been applied for medical use cases.",
    "Compare the performance of random forests and XGBoost, and explain which algorithm performs better in which case.",
    "Is there practical research where Uniform Convergence property has been used to compare different learning algorithms?",
    "Datasets where the framework of PAC learning is demonstrated to work.",
    "Are there any pitfalls of using cross-validation for model selection?",
    "Theoretical justifications for effects of different hyper-parameters on the performance of random forest?",
    "How is central limit theorem useful for analysis of model performance and convergence of learning algorithms?",
    "Use of Hilbert space in machine learning.",
    "How does high-dimensional data affect different learning algorithms.",
    "Which learning algorithms have been used to model brain activity in humans?",
    "Algorithms used for ranking in recommender systems.",
    "Why is normalization needed for co-occurrence matrices in recommendation system?",
    "Practical datasets which compare the performance of PCA and kernel PCA.",
    "Is regularization really needed in neural networks?",
    "Any techniques which have the effect of implicit regularization in neural networks?",
    "What does batch normalization really do mathematically in neural networks?",
    "Comparison of fraud detection methods in banking.",
    "Mathematical understanding of weight initialization methods in neural networks.",
    "Practical cases where less powerful models achieve better performance than more powerful models."
]

len(query_list)

20

In [103]:
from tqdm import tqdm

In [110]:
base_encoder_evaluations = []

for query in tqdm(query_list):
    try:
        base_encoder_evaluations.append(evaluator_base.evaluate_retrieval(query))
    except:
        pass

100%|██████████| 20/20 [02:26<00:00,  7.35s/it]


In [111]:
base_encoder_evaluations[:3]

[{'query': 'What are some breakthroughs in theoretical understanding of neural networks?',
  'passages': ('A short account is  given of various  investigations of neural  network  properties,  beginning  with  the  classic  work of McCulloch  & Pitts.  Early work on neurodynamics and statistical mechanics, analogies with  magnetic materials, fault tolerance via parallel distributed processing,  memory, learning,  and pattern recognition,  is described.',
   'With a goal of understanding what drives generalization in deep networks, we consider several recently suggested explanations, including norm-based control, sharpness and robustness. We study how these measures can ensure generalization, highlighting the importance of scale normalization, and making a connection between sharpness and PAC-Bayes theory.  We then investigate how well the measures explain different observed phenomena.',
   'In this paper we investigate an average-case model of concept learning, and  give results that p

In [113]:
finetuned_encoder_evaluations = []

for query in tqdm(query_list):
    try:
        finetuned_encoder_evaluations.append(evaluator_finetuned.evaluate_retrieval(query))
    except:
        pass

100%|██████████| 20/20 [02:09<00:00,  6.50s/it]


In [114]:
finetuned_encoder_evaluations[:3]

[{'query': 'What are some breakthroughs in theoretical understanding of neural networks?',
  'passages': ('A short account is  given of various  investigations of neural  network  properties,  beginning  with  the  classic  work of McCulloch  & Pitts.  Early work on neurodynamics and statistical mechanics, analogies with  magnetic materials, fault tolerance via parallel distributed processing,  memory, learning,  and pattern recognition,  is described.',
   'The success of Deep Learning and its potential use in many safety-critical\n  applications has motivated research on formal verification of Neural Network\n  (NN) models. Despite the reputation of learned NN models to behave as black\n  boxes and the theoretical hardness of proving their properties, researchers\n  have been successful in verifying some classes of models by exploiting their\n  piecewise linear structure and taking insights from formal methods such as\n  Satisifiability Modulo Theory. These methods are however still 

In [115]:
print(len(base_encoder_evaluations))
print(len(finetuned_encoder_evaluations))

20
12


##### 8 cases where some error was there, fix later

In [116]:
avg_relevance_base_encoder = sum([sum(ev['rel_scores'])/3 for ev in base_encoder_evaluations]) / len(base_encoder_evaluations)
avg_relevance_base_encoder

0.7666666666666666

In [117]:
avg_relevance_finetuned_encoder = sum([sum(ev['rel_scores'])/3 for ev in finetuned_encoder_evaluations]) / len(finetuned_encoder_evaluations)
avg_relevance_finetuned_encoder

3.2499999999999996

### average retrieval relevance scores of the finetuned encoder is quite better than the base encoder

In [119]:
avg_precision_base_encoder = sum([sum(ev['bin_scores'])/3 for ev in base_encoder_evaluations]) / len(base_encoder_evaluations)
avg_precision_base_encoder

0.05

In [120]:
avg_precision_finetuned_encoder = sum([sum(ev['bin_scores'])/3 for ev in finetuned_encoder_evaluations]) / len(finetuned_encoder_evaluations)
avg_precision_finetuned_encoder

0.3055555555555555

### average precision scores of the finetuned encoder is also quite better than the base encoder, therefore we can use the finetuned encoder for the retrieval part of our RAG

### generation using LLM

In [122]:
generation_pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1000
)
generation_llm = HuggingFacePipeline(pipeline=generation_pipe)

Device set to use cuda:0


In [135]:
# Generate a response based on the combined top 3 abstracts
query = query_list[0]
retrieved_docs, retrieved_scores = zip(*evaluator_finetuned.retrieve(query = "Which learning algorithms have been used to model brain activity in humans?", k=3))
context = f""
for idx, d in enumerate(retrieved_docs):
    context += f"Document {idx+1}: {d} \n"
prompt = f""" Query: {query} \n Prompt: Use the context which is provided to answer the given query.  The context is a set of technical
papers which are not suitable for clear understanding for a common person. Summarize your answer and in such a way that an undergraduate student
can also understand it. I also need the summary in a structured way, like someone would write a literature review. \n Context: {context}"""
print(prompt)

 Query: What are some breakthroughs in theoretical understanding of neural networks? 
 Prompt: Use the context which is provided to answer the given query.  The context is a set of technical
papers which are not suitable for clear understanding for a common person. Summarize your answer and in such a way that an undergraduate student
can also understand it. I also need the summary in a structured way, like someone would write a literature review. 
 Context: Document 1: Neuroscience studies of human decision-making abilities commonly involve
subjects completing a decision-making task while BOLD signals are
recorded using fMRI. Hypotheses are tested about which brain regions
mediate the effect of past experience, such as rewards, on future
actions. One standard approach to this is model-based fMRI data
analysis, in which a model is fitted to the behavioral data, i.e., a
subject's choices, and then the neural data are parsed to find brain
regions whose BOLD signals are related to the mode

In [136]:
response = generation_llm(prompt)
print(f"Generated Response: {response}")

Generated Response:  Query: What are some breakthroughs in theoretical understanding of neural networks? 
 Prompt: Use the context which is provided to answer the given query.  The context is a set of technical
papers which are not suitable for clear understanding for a common person. Summarize your answer and in such a way that an undergraduate student
can also understand it. I also need the summary in a structured way, like someone would write a literature review. 
 Context: Document 1: Neuroscience studies of human decision-making abilities commonly involve
subjects completing a decision-making task while BOLD signals are
recorded using fMRI. Hypotheses are tested about which brain regions
mediate the effect of past experience, such as rewards, on future
actions. One standard approach to this is model-based fMRI data
analysis, in which a model is fitted to the behavioral data, i.e., a
subject's choices, and then the neural data are parsed to find brain
regions whose BOLD signals are